In [1]:
import os
import torch
import random
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

resnet_type = "resnet18"
image_size = 224
batch_size = 8
num_epochs = 50
learning_rate = 0.001
num_classes = 4

train_dir = "../data/Train/"
test_dir = "../data/Test"


# Define the transformations for train data before entering the neural network
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),                                    # Resize images to 224x224 pixels
    transforms.RandomCrop(size=image_size),                                         # Randomly crop and resize images to 224x224 pixels
    transforms.RandomHorizontalFlip(),                                              # Randomly flip the images horizontally
    transforms.RandomRotation(30),                                                  # Randomly rotate the images by 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),                                                          # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])                 # Normalize the images
])

# Define the transformations for test data before entering the neural network
transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),                     # Resize images to 224x224 pixels
    transforms.ToTensor(),                                           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

# Load the training and test datasets
train_dataset = ImageFolder(train_dir, transform=transform_train)
# val_dataset = ImageFolder(test_dir, transform=transform_test)
# test_dataset = ImageFolder(test_dir, transform=transform_test)

# Create DataLoaders for training, validation, and test datasets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Split the dataset into training, validation, and test sets
dataset_size = len(train_dataset)
train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size, test_size])


# Create DataLoaders for managing the data batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Resnet model
model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

def train(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_loader)


def test(model, criterion, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(test_loader), accuracy


best_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = test(model, criterion, test_loader, device)

    scheduler.step(test_loss)  

    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

    
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), 'best_model_1.pth')

        
# data = {
#     "Model type": resnet_type,
#     "Dataset use": os.path.basename(train_dir),
#     "Image Resize": str(image_size)+"*"+str(image_size),
#     "Epochs": num_epochs,
#     "Learning rate": learning_rate,
#     "Batch size": batch_size,
#     "Train Accuracy": f"{train_acc:.2f}",
#     "Validation accuracy": f"{val_acc:.2f}",
#     "Test Accuracy": f"{test_acc:.2f}",
# }

# # Check if the CSV file already exists
# if os.path.isfile("model_data.csv"):
#     existing_data = pd.read_csv("model_data.csv")
#     new_data = pd.concat([existing_data, pd.DataFrame(data, index=[0])], ignore_index=True)

# else:
#     new_data = pd.DataFrame(data, index=[0])

# # Save the updated DataFrame to CSV
# new_data.to_csv("model_data.csv", index=False)


# # Plot training and validation accuracy
# fig, ax = plt.subplots()
# ax.set(xlabel='Epoch', ylabel='Accuracy', title="Training and Validation Accuracy")

# epochs = range(1, num_epochs + 1)
# plt.plot(epochs, train_loss, label='Training Accuracy')
# plt.plot(epochs, test_loss, label='Validation Accuracy')
# plt.legend()
# plt.show()




cuda


/home/pans/miniconda3/envs/miw/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pans/miniconda3/envs/miw/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50: Train Loss: 1.4953, Test Loss: 1.3235, Test Accuracy: 48.72%
Epoch 2/50: Train Loss: 1.2298, Test Loss: 0.8946, Test Accuracy: 61.54%
Epoch 3/50: Train Loss: 1.0488, Test Loss: 0.8352, Test Accuracy: 69.23%
Epoch 4/50: Train Loss: 1.0077, Test Loss: 0.7926, Test Accuracy: 64.10%
Epoch 5/50: Train Loss: 0.9793, Test Loss: 0.6287, Test Accuracy: 76.92%
Epoch 6/50: Train Loss: 0.8877, Test Loss: 1.0780, Test Accuracy: 58.97%
Epoch 7/50: Train Loss: 0.8402, Test Loss: 0.6078, Test Accuracy: 82.05%
Epoch 8/50: Train Loss: 0.6648, Test Loss: 0.8462, Test Accuracy: 69.23%
Epoch 9/50: Train Loss: 0.6102, Test Loss: 0.4774, Test Accuracy: 79.49%
Epoch 10/50: Train Loss: 0.6183, Test Loss: 0.6071, Test Accuracy: 84.62%
Epoch 11/50: Train Loss: 0.6422, Test Loss: 0.9724, Test Accuracy: 69.23%
Epoch 12/50: Train Loss: 0.6077, Test Loss: 0.5442, Test Accuracy: 82.05%
Epoch 13/50: Train Loss: 0.5313, Test Loss: 0.3689, Test Accuracy: 84.62%
Epoch 14/50: Train Loss: 0.5270, Test Loss: 0.2